In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
ls

gdrive/  sample_data/


In [3]:
import os
os.chdir("/content/gdrive/My Drive/prefixtune")

In [4]:
ls

answer/  check.py  iocollect.py  output.txt  __pycache__/  requirements.txt  zipsrc.py
bleu.py  data/     output/       output.zip  README.md     zipout.py


In [5]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 92.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.9/466.9 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
import torch
torch.cuda.is_available()

True

In [ ]:
!pip install --upgrade peft transformers datasets

## Experiments with
1. epoches: 3-10
2. learning rate: 1e-2 to 1e-5
3. virtual tokens: 5-20
4. prefix projection: T/F
5. training dataset: e2e_nlg_cleaned or data/train.txt
6. Beam Search (num_beams): Higher values can improve output quality but increase computational cost. Experiment with 5-10.
7. Sampling Parameters: Adjust top_p and temperature to balance diversity and coherence.
8. Deterministic Output: For consistent evaluation, consider setting do_sample=False



In [81]:
!python answer/default.py -i data/input/small.txt -e=3 -v=10 > output.txt

2024-10-20 21:37:06.646469: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-20 21:37:06.665717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-20 21:37:06.671663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-20 21:37:07.869510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Found modelfile data/peft.pt. Starting decoding.
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/peft/peft_model.py:1755: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings

In [62]:
from answer.default import *
import os, sys

basemodel = 'distilgpt2'
table_to_text = TableToText("peft", basemodel=basemodel)
model = AutoModelForCausalLM.from_pretrained(basemodel)
model = PeftModel.from_pretrained(model, 'data/peft.pt')
model = model.to(device)
decoder_output = table_to_text.decode(model, 'data/input/small.txt')
print("\n".join(decoder_output))


1it [00:00,  2.26it/s]

Generated text: '  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  The Alimentum is located in the price range.  It is'


2it [00:00,  2.39it/s]

Generated text: '  The Alimentum is located near the city centre near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near'


3it [00:01,  2.45it/s]

...

Generated text: '  In the riverside area near the Crown Plaza Hotel there is a place called Alimentum. It is located near the Crown Plaza Hotel. It is located near the Crown Plaza Hotel.  It is located near the Crown Plaza Hotel.  It is'


9it [00:03,  2.48it/s]

Generated text: '  Aromi is a place with a customer rating of 1 out of 5. It is located in the riverside area. It is located in the riverside area. It has a customer rating of 1 out of 5. It is located in the'


10it [00:04,  2.47it/s]

Generated text: '  Located in the riverside area, there is a place to eat.  Located in the riverside area, there is a place to eat Chinese food.  It has a customer rating of 3 out of 5.  It is located in the rivers'
0||  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  The Alimentum is located in the price range.  It is
1||  The Alimentum is located near the city centre near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near
2||  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  It has a customer rating of 1 out of 5.  It has a customer rating of 1 out of 5.  It is
3||  The Alimentum is located near the city centre near the Crown Plaza Hotel.  Located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum i

In [66]:
print("\n".join(decoder_output))

0||  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  The Alimentum is located in the price range.  It is
1||  The Alimentum is located near the city centre near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near
2||  The Alimentum is located in the city centre.  The Alimentum is located in the city centre.  It has a customer rating of 1 out of 5.  It has a customer rating of 1 out of 5.  It is
3||  The Alimentum is located near the city centre near the Crown Plaza Hotel.  Located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.
4||  The Alimentum is located in the riverside.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is located near the Crown Plaza Hotel.  The Alimentum is loca

In [75]:
outputfile='output.txt'

with open(outputfile, 'w') as out_f:
    for i in decoder_output:
        out_f.write(f"\n{i}")

print(f"Decoding completed. Outputs saved to {outputfile}.")

Decoding completed. Outputs saved to output.txt.


In [78]:
!python bleu.py -t data/reference/small.out -o output.txt

bleu score: 19.8489116815088
